There are two ways of collecting and analyzing NGINX logs in AWS:

1. Download all the logentries from Cloudwatch filtered on `StreamPrefix='application/nginx'`` and analyze them locally
2. Execute a query with boto3 and analyze the results

In [6]:
%pip install boto3 prettytable pandas


[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
import boto3
import subprocess
import json
import os
import time
import re
import random
import pandas as pd
from datetime import datetime, timezone, timedelta
from concurrent.futures import ThreadPoolExecutor, as_completed

import sys
sys.path.append('..')
from awsutils import *

In [8]:

# Use the function with your profile to get a session
aws_session = set_aws_credentials('acl-production', 'eu-central-1')

# Create clients using the session
sts_client = aws_session.client('sts')
logs_client = aws_session.client('logs')

# Example usage of the clients
account_id = sts_client.get_caller_identity()["Account"]
print("Current AWS Account ID:", account_id)

Current AWS Account ID: 707785685172


# Analyze logs with boto3

In [9]:
def fetch_slices(logs_client, log_group, base_query, start_date, end_date, file_name, slices=100):
    slice_duration = (end_date - start_date) / slices
    query = base_query + f" | limit 10000"

    future_to_task = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        for i in range(slices):
            # Calculate the start and end timestamps for the current time slice
            start_timestamp = int(start_date + (i * slice_duration))
            end_timestamp = int(start_date + ((i + 1) * slice_duration))
        
            # print time slice in human readable format (original format is epoch time in seconds)
            print(f"Time slice {i}: {datetime.fromtimestamp(start_timestamp)} - {datetime.fromtimestamp(end_timestamp)}")

            future_to_task.append(executor.submit(
                cloudwatch_query, logs_client, log_group, query, start_timestamp, end_timestamp))
            
        for future in as_completed(future_to_task):
            # print the number of log entries in each time slice
            print("Number of log entries:", len(future.result()))

            # append results from each time slice to a file
            with open(file_name, 'a') as f:
                f.write(json.dumps(future.result()))
                f.write('\n')
            

In [10]:


log_group = 'projects-main'
#log_group =  'api_proxy-main'

base_query = f"fields @timestamp, @message | filter @logStream like 'application/nginx'"
start_date = exact_date(2024, 1, 24)
end_date = exact_date(2024, 1, 25)


# clear the file before appending new data
file_name='log_data_20240124.json'
with open(file_name, 'w') as f:
    f.write('')
fetch_slices(logs_client, log_group, base_query, start_date, end_date, file_name, slices=400)



Time slice 0: 2024-01-24 00:00:00 - 2024-01-24 00:03:36
Time slice 1: 2024-01-24 00:03:36 - 2024-01-24 00:07:12
Time slice 2: 2024-01-24 00:07:12 - 2024-01-24 00:10:48
Time slice 3: 2024-01-24 00:10:48 - 2024-01-24 00:14:24
Time slice 4: 2024-01-24 00:14:24 - 2024-01-24 00:18:00
Time slice 5: 2024-01-24 00:18:00 - 2024-01-24 00:21:36
Time slice 6: 2024-01-24 00:21:36 - 2024-01-24 00:25:12
Time slice 7: 2024-01-24 00:25:12 - 2024-01-24 00:28:48
Time slice 8: 2024-01-24 00:28:48 - 2024-01-24 00:32:24
Time slice 9: 2024-01-24 00:32:24 - 2024-01-24 00:36:00
Time slice 10: 2024-01-24 00:36:00 - 2024-01-24 00:39:36
Time slice 11: 2024-01-24 00:39:36 - 2024-01-24 00:43:12
Time slice 12: 2024-01-24 00:43:12 - 2024-01-24 00:46:48
Time slice 13: 2024-01-24 00:46:48 - 2024-01-24 00:50:24
Time slice 14: 2024-01-24 00:50:24 - 2024-01-24 00:54:00
Time slice 15: 2024-01-24 00:54:00 - 2024-01-24 00:57:36
Time slice 16: 2024-01-24 00:57:36 - 2024-01-24 01:01:12
Time slice 17: 2024-01-24 01:01:12 - 2024

ClientError: An error occurred (ExpiredTokenException) when calling the GetQueryResults operation: The security token included in the request is expired

In [11]:
def normalize_request_path(request):
    # Remove "HTTP/1.1" if it exists at the end
    request = re.sub(r' HTTP/1.1$', '', request)

    # Regular expression to identify and replace numeric IDs in the request path
    return re.sub(r'[/=]\d+', '{id}', request)

def aggregate_requests(aggregated_data, data):
    for entry in data:
        try:
            request = entry["@message"]["request"]
            normalized_request = normalize_request_path(request)
            request_time = int(float(entry["@message"]["request_time"]) * 1000)  # Convert to milliseconds

            if normalized_request in aggregated_data:
                aggregated_data[normalized_request]['count'] += 1
                aggregated_data[normalized_request]['total_duration'] += request_time
            else:
                aggregated_data[normalized_request] = {'count': 1, 'total_duration': request_time}
        except KeyError:
            continue
        except Exception as e:
            print("Error processing entry:", entry)
            print(e)
            continue


In [12]:
# load the json file with log entries into a list
aggregated_data = {}

log_files = ['log_data_20240122.json', 'log_data_20240123.json', 'log_data_20240124.json']
item_count=0
        
for fname in log_files:
    with open(fname, 'r') as f:
        for line in f:
            data = json.loads(line)
            item_count += len(data)
            print(f'\rnumber of log entries: {item_count}', end="")
            parse_json_messages(data)
            aggregate_requests(aggregated_data, data)



number of log entries: 318831Error processing entry: {'@timestamp': '2024-01-22 01:15:19.621', '@message': '2024/01/22 01:15:19 [error] 52#0: *3 connect() failed (111: Connection refused) while connecting to upstream, client: 10.185.255.33, server: _, request: "GET /health-check-elb HTTP/1.1", upstream: "http://127.0.0.1:9393/health-check-elb", host: "10.185.78.12:2000"\r', '@ptr': 'CmAKHgoaNzA3Nzg1Njg1MTcyOnByb2plY3RzLW1haW4QBhI6GhgCBlqsJYEAAAAAA00aNQAGWtwUwAAAAhIgASjHuun10jEwq5Xz9dIxOO5OQPjijQNI/+EfUNKsHxgAIAEQ6kYYAQ=='}
string indices must be integers, not 'str'
Error processing entry: {'@timestamp': '2024-01-22 01:15:19.579', '@message': '2024/01/22 01:15:19 [error] 52#0: *1 connect() failed (111: Connection refused) while connecting to upstream, client: 10.185.254.157, server: _, request: "GET /health-check-elb HTTP/1.1", upstream: "http://127.0.0.1:9393/health-check-elb", host: "10.185.78.12:2000"\r', '@ptr': 'CmAKHgoaNzA3Nzg1Njg1MTcyOnByb2plY3RzLW1haW4QBhI6GhgCBlqsJYEAAAAAA00aNQ

In [13]:


# Print the top 10 requests by total duration in a table
df = pd.DataFrame.from_dict(aggregated_data, orient='index')
df['avg_duration'] = round(df['total_duration'] / df['count'], 2)
df = df.sort_values('total_duration', ascending=False)
df = df.reset_index()
df.columns = ['request', 'count', 'total_duration', 'avg_duration']

from prettytable import PrettyTable
x = PrettyTable()
x.field_names = ["Request", "Count", "Total Duration", "Average Duration"]
x.align["Request"] = "l"
for index, row in df.head(50).iterrows():
    # truncate the request path to 80 characters    
    request = row['request'][:80] + '...' if len(row['request']) > 50 else row['request']
    x.add_row([request, row['count'], timedelta(milliseconds=row['total_duration']), row['avg_duration']])
print(x)




+-------------------------------------------------------------------------------------+---------+-------------------------+------------------+
| Request                                                                             |  Count  |      Total Duration     | Average Duration |
+-------------------------------------------------------------------------------------+---------+-------------------------+------------------+
| GET /api/v3/mitigations{id}?org_id{id}                                              | 3148561 | 3 days, 22:08:31.237000 |      107.64      |
| GET /api/v3/projects{id}/collaborators?org_id{id}                                   |  360074 |  2 days, 2:18:55.610000 |      503.05      |
| GET /api/v3/objectives{id}/risks?org_id{id}&fields%5Brisks%5D=title%2Cdescriptio... |  724481 |  1 day, 7:30:19.241000  |      156.55      |
| GET /api/v3/controls{id}/control_performance_schedules?org_id{id}&fields%5Bcontr... |  407205 |     15:25:18.982000     |      136.34      |

# Collect logs from Cloudwatch

In [ ]:
# select 'projects-main' log group and collect log streams starting with 'application/nginx/'
# get log entries based on a time frame ginven in parameters

def get_log_entries(logs_client, log_group_name, log_stream_prefix, start_time, end_time):
    paginator = logs_client.get_paginator('filter_log_events')
    response_iterator = paginator.paginate(
        logGroupName=log_group_name,
        logStreamNamePrefix=log_stream_prefix,
        startTime=start_time,
        endTime=end_time,
        #PaginationConfig={'MaxItems': 1000}
    )
    log_events = []
    for response in response_iterator:
        log_events += response['events']
    return log_events

log_group_name = 'projects-main'
log_stream_name_prefix = 'application/nginx'

start_time = int(datetime(2024, 1, 1, 0, 0, 0, tzinfo=timezone.utc).timestamp() * 1000)  # 1st Jan 2024
end_time = int(datetime(2024, 1, 3, 0, 0, 0, tzinfo=timezone.utc).timestamp() * 1000)   # 2nd Jan 2024

log_events = get_log_entries(logs_client, log_group_name, log_stream_name_prefix, start_time, end_time)
print("Number of log events:", len(log_events))

# print the top 100 entries
for log_event in log_events[:100]:
    print(log_event['message'])


In [15]:
with open('log_data_20240122.json', 'r') as f:
    for line in f:
        data = json.loads(line)
        item_count += len(data)
        print(f'\rnumber of log entries: {item_count}', end="")
        parse_json_messages(data)

number of log entries: 12925924

In [17]:
print(json.dumps(data[0], indent=4, sort_keys=True))

{
    "@message": {
        "ACL_Cloudfront": "",
        "body_bytes_sent": "2787",
        "bytes_sent": "3816",
        "client": "10.185.255.192",
        "host": "projects-eu.highbond.com",
        "method": "GET",
        "referrer": "",
        "remote_user": "chris@satorigroup.in",
        "request": "GET /api/v3/controls/1696230/control_performance_schedules?org_id=3983&fields%5Bcontrol_performance_schedules%5D=enabled%2Cfrequency_type%2Csent%2Cfrequency_days%2Cfrequency_run_at%2Cfrequency_starts_on%2Cperformer_contact_emails%2Cfrequency_timezone%2Ccreated_at%2Cupdated_at%2Cfrequency_run_at%2Cfrequency_timezone_offset HTTP/1.1",
        "request_length": "4097",
        "request_time": "0.120",
        "scheme": "http",
        "status": "200",
        "time_local": "22/Jan/2024:23:00:00 +0000",
        "upstream_connect_time": "0.004",
        "upstream_header_time": "0.120",
        "upstream_response_time": "0.120",
        "upstream_status": "200",
        "user_agent": "p